In [ ]:
#Start with SLU-01
station_id='SLU-01'
print('The Station {}'.format(station_id)+' has {} docks in total.'.format(station.loc[station_id,'install_dockcount']))

#Station by station
extract = trip.loc[(trip.from_station_id==station_id) | (trip.to_station_id==station_id),:]
def incrementation(row):
    if (row['from_station_id']==station_id)&(row['to_station_id']==station_id):
        return int(0)
    if (row['from_station_id']==station_id):
        return int(-1)
    if (row['to_station_id']==station_id):
        return int(1)
extract['incrementation'] = trip.apply(incrementation, axis=1)
extract = extract.set_index('trip_id')

#Start and Stop
temp1 = extract.loc[(extract.incrementation==0),['starttime','stoptime','bikeid','to_station_id','incrementation']]
instanteanous_variation = pd.DataFrame(columns=['trip_id','time', 'bikeid', 'destination_id', 'incrementation'])
for i in range(temp1.shape[0]):
    #-1
    serie1 = dict(trip_id=temp1.index[i],bikeid=temp1.bikeid.values[i],destination_id=temp1.to_station_id.values[i])
    serie1['incrementation'] = -1
    serie1['time'] = temp1.starttime.values[i]
    
    #+1
    serie2 = dict(trip_id=temp1.index[i],bikeid=temp1.bikeid.values[i],destination_id=temp1.to_station_id.values[i])
    serie2['incrementation'] = 1
    serie2['time'] = temp1.stoptime.values[i]
    
    instanteanous_variation = instanteanous_variation.append(serie1, ignore_index=True)
    instanteanous_variation = instanteanous_variation.append(serie2, ignore_index=True)
instanteanous_variation = instanteanous_variation.set_index('trip_id')
instanteanous_variation.index = instanteanous_variation.index.astype(int) 

#Stop
temp2 = extract.loc[(extract.incrementation==1.0),['stoptime','bikeid','from_station_id','incrementation']]
temp2.columns=['time','bikeid','destination_id','incrementation']
instanteanous_variation=instanteanous_variation.append(temp2)

#Start
temp3 = extract.loc[(extract.incrementation==-1.0),['starttime','bikeid','to_station_id','incrementation']]
temp3.columns=['time','bikeid','destination_id','incrementation']
instanteanous_variation=instanteanous_variation.append(temp3)

#Sort by time before doing cumulative
instanteanous_variation.time = pd.to_datetime(instanteanous_variation.time)
instanteanous_variation = instanteanous_variation.sort_values('time')

#Computation of the total cumulative variation
instanteanous_variation['total_variation'] = instanteanous_variation['incrementation'].cumsum()

#Display DF
display(instanteanous_variation.head())

# Visualization
instanteanous_variation.plot(x='time',y='total_variation',figsize=(30,10), grid=True, fontsize=15)
plt.show()